In [1]:
import numpy as np
import pandas as pd
from medpy.io import load
import os
import cv2
from sklearn.model_selection import train_test_split

# Import data

In [2]:
path="./NN/waa"
carpet="Ultrasound_minFrame"
AM_FM_images={}
for _,i,_ in os.walk(path):
    if len(i)==100:
        subjects=i

perc_train=0.75
names_train,names_test=train_test_split(subjects, train_size=perc_train, 
                                        test_size=1-perc_train, random_state=3)
names_val,names_test=train_test_split(names_test, train_size=15/25, 
                                        test_size=10/25, random_state=3)

print(len(names_train))
print(len(names_val))
print(len(names_test))

75
15
10


In [3]:
def import_data(path, subjects, reshape):
    # This function loads the csv files and images. Moreover, it removes data that do not have a value for muscles.
    
    x_img = [] #Original image
    x_ctg = pd.DataFrame() #Categorical data
    y = pd.DataFrame() #csv files
    
    i = 1
    for subject in subjects:
        print(subject,'number',i)
        carpet="Ultrasound_minFrame"
        
        # Load csv files
        values = pd.read_csv(os.path.join(path, subject, 'thickness.csv')) # Thickness
        ctg_data = pd.read_csv(os.path.join(path, subject, 'categorical_values.csv')) # Categorical data (extremity and position)
        
        # Remove NAN data
        index = values['Muscle'].index[values['Muscle'].apply(np.isnan)] # Index where there is no value for muscle
        values.dropna(subset = ["Muscle"], inplace=True)
        x_ctg = x_ctg.append(ctg_data.drop(index=index), ignore_index=True)
        y = y.append(values, ignore_index=True)
        
        # Load images
        directory = os.path.join(path, subject, carpet)
        for _,_,images_name in os.walk(directory):
            j=0
            total=0
            for image_name in images_name:
                if j not in index: 
                    total += 1
                    image, _ = load(os.path.join(directory, image_name))
                    image = image[201:801, 31:745, 0, 0].T/255
                    if reshape:
                        image = cv2.resize(image, (256, 304))
                    x_img.append(image)
                j+=1
        print('# images used: %i' %total)
        print(list(index))
        print('-'*40)
        i+=1
        
    x_img = np.array(x_img, dtype=np.float32)
    dim = x_img.shape
    dim = np.append(dim, 1) # We add a fourth dimension which represents the number of channels
    
    # Converting categorical data to numerical data
    cleanup = {
        'Extremity': {'LL': 1, 'UL': 2, 'LA': 3, 'UA': 4},
        'Position': {'AD': 1, 'AC': 2, 'AP': 3, 'LD': 4, 'LP': 5, 'LC': 6, 'MC': 7, 'MD': 8, 'MP': 9, 'PC': 10, 'PD': 11, 'PP': 12}
    }
    x_ctg.replace(cleanup, inplace=True)
    
    return x_img.reshape(dim), x_ctg.drop(columns=['Name']), y.drop(columns=['Name'])

In [4]:
x_img_train, x_ctg_train, y_train=import_data(path, names_train[:75], reshape=True)
print('shape x_img_train', x_img_train.shape)
print('shape x_ctg_train', x_ctg_train.shape)
print('shape y_train', y_train.shape)

MULTIS088-1 number 1
# images used: 45
[18, 23, 24]
----------------------------------------
MULTIS019-1 number 2
# images used: 48
[]
----------------------------------------
MULTIS067-1 number 3
# images used: 45
[21, 46, 47]
----------------------------------------
MULTIS049-1 number 4
# images used: 48
[]
----------------------------------------
MULTIS052-1 number 5
# images used: 46
[46, 47]
----------------------------------------
MULTIS032-1 number 6
# images used: 47
[46]
----------------------------------------
MULTIS007-1 number 7
# images used: 48
[]
----------------------------------------
MULTIS015-1 number 8
# images used: 44
[9, 30, 40, 45]
----------------------------------------
MULTIS033-1 number 9
# images used: 47
[46]
----------------------------------------
MULTIS090-1 number 10
# images used: 46
[46, 47]
----------------------------------------
MULTIS013-1 number 11
# images used: 46
[46, 47]
----------------------------------------
MULTIS060-1 number 12
# images

In [5]:
x_img_val, x_ctg_val, y_val = import_data(path, names_val, reshape=True)
print('shape x_img_val', x_img_val.shape)
print('shape x_ctg_val', x_ctg_val.shape)
print('shape y', y_val.shape)

MULTIS036-1 number 1
# images used: 48
[]
----------------------------------------
MULTIS098-1 number 2
# images used: 45
[25, 46, 47]
----------------------------------------
MULTIS014-1 number 3
# images used: 48
[]
----------------------------------------
MULTIS100-1 number 4
# images used: 45
[27, 46, 47]
----------------------------------------
MULTIS044-1 number 5
# images used: 46
[46, 47]
----------------------------------------
MULTIS085-1 number 6
# images used: 44
[24, 25, 46, 47]
----------------------------------------
MULTIS075-1 number 7
# images used: 47
[46]
----------------------------------------
MULTIS017-1 number 8
# images used: 47
[26]
----------------------------------------
MULTIS011-1 number 9
# images used: 48
[]
----------------------------------------
MULTIS081-1 number 10
# images used: 47
[47]
----------------------------------------
MULTIS047-1 number 11
# images used: 46
[46, 47]
----------------------------------------
MULTIS095-1 number 12
# images us

In [50]:
x_img_test, x_ctg_test, y_test = import_data(path, names_test, reshape=False)
print('shape x_test1', x_img_test.shape)
print('shape x_test1', x_ctg_test.shape)
print('shape y', y_test.shape)

MULTIS061-1 number 1
# images used: 48
[]
----------------------------------------
MULTIS025-1 number 2
# images used: 48
[]
----------------------------------------
MULTIS073-1 number 3
# images used: 48
[]
----------------------------------------
MULTIS006-1 number 4
# images used: 47
[26]
----------------------------------------
MULTIS077-1 number 5
# images used: 45
[25, 46, 47]
----------------------------------------
MULTIS084-1 number 6
# images used: 47
[46]
----------------------------------------
MULTIS099-1 number 7
# images used: 46
[46, 47]
----------------------------------------
MULTIS008-1 number 8
# images used: 48
[]
----------------------------------------
MULTIS069-1 number 9
# images used: 48
[]
----------------------------------------
MULTIS010-1 number 10
# images used: 46
[41, 42]
----------------------------------------
shape x_test1 (471, 714, 600, 1)
shape x_test1 (471, 2)
shape y (471, 3)


# Pre-process

In [7]:
# Normalize the data: 

y_info = y_train.copy()

# Z-score normalization
y1_train = (y_train - y_train.mean())/y_train.std()
y1_val = (y_val - y_train.mean())/y_train.std()
y1_test = (y_test - y_train.mean())/y_train.std()


# Min-max normalization
y2_train = (y_train - y_train.min())/(y_train.max() - y_train.min())
y2_val = (y_val - y_train.min())/(y_train.max() - y_train.min())
y2_test = (y_test - y_train.min())/(y_train.max() - y_train.min())

# Decimal scaling normalization
y3_train = y_train/np.array([10,100,100])
y3_val = y_val/np.array([10,100,100])
y3_test = y_test/np.array([10,100,100])

In [8]:
def remove_outliers(x_img, x_ctg, y, y1, y2, y3):

    z_score = y1
    
    index = z_score.index[((abs(z_score)<3).apply(np.sum,axis=1))==3].tolist()
    index_images = np.sum(abs(np.array(y1))<3, axis=1)==3
    
    x_ctg, y, y1, y2, y3 = x_ctg.loc[index], y.loc[index], y1.loc[index], y2.loc[index], y3.loc[index]
    x_img_new = x_img[index_images]
    
    return x_img_new, np.array(x_ctg), np.array(y), np.array(y1), np.array(y2), np.array(y3)

In [9]:
from sklearn.utils import shuffle

print('BEFORE')
print('x_img_train', x_img_train.shape)
print('x_ctg_train', x_ctg_train.shape)
print('y_train: ', y_train.shape)
print('x_img_val', x_img_val.shape)
print('x_ctg_val', x_ctg_val.shape)
print('y_val: ', y_val.shape)
print('x_test', x_img_test.shape)
print('x_test', x_ctg_test.shape)
print('y_test: ', y_test.shape)
print('-'*50)

# Remove outliers
x_img_train, x_ctg_train, y_train , y1_train, y2_train, y3_train = remove_outliers(x_img_train, x_ctg_train, 
                                                                                   y_train, y1_train, y2_train, y3_train)

x_img_val, x_ctg_val, y_val, y1_val, y2_val, y3_val = remove_outliers(x_img_val, x_ctg_val, 
                                                               y_val, y1_val, y2_val, y3_val)

x_img_test, x_ctg_test, y_test, y1_test, y2_test, y3_test = remove_outliers(x_img_test ,x_ctg_test, 
                                                                            y_test, y1_test, y2_test, y3_test)

# Shuffle training data
x_img_train, x_ctg_train, y_train, y1_train, y2_train, y3_train = shuffle(x_img_train, x_ctg_train, 
                                                             y_train, y1_train, y2_train, y3_train, random_state=0)

print('AFTER')
print('x_img_train', x_img_train.shape)
print('x_ctg_train', x_ctg_train.shape)
print('y_train: ', y_train.shape)
print('x_img_val', x_img_val.shape)
print('x_ctg_val', x_ctg_val.shape)
print('y_val: ', y_val.shape)
print('x_test', x_img_test.shape)
print('x_test', x_ctg_test.shape)
print('y_test: ', y_test.shape)

BEFORE
x_img_train (3477, 304, 256, 1)
x_ctg_train (3477, 2)
y_train:  (3477, 3)
x_img_val (698, 304, 256, 1)
x_ctg_val (698, 2)
y_val:  (698, 3)
x_test (471, 304, 256, 1)
x_test (471, 2)
y_test:  (471, 3)
--------------------------------------------------
AFTER
x_img_train (3370, 304, 256, 1)
x_ctg_train (3370, 2)
y_train:  (3370, 3)
x_img_val (671, 304, 256, 1)
x_ctg_val (671, 2)
y_val:  (671, 3)
x_test (463, 304, 256, 1)
x_test (463, 2)
y_test:  (463, 3)


# Deep Learning

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, activations, regularizers, Input
import models_and_functions_phase1 as cc

In [11]:
# Here you have two options
# You can use the function called'network' if you want to build a model that does not include the categorical. 
# Or you can use the function called 'network_categorical'

snn_model1 = cc.network_categorical(input_shape = x_img_train.shape[-3:]) # Z-score

snn_model2 = cc.network_categorical(input_shape = x_img_train.shape[-3:]) # Z-score

snn_model3 = cc.network_categorical(input_shape = x_img_train.shape[-3:]) # min-max

snn_model4 = cc.network_categorical(input_shape = x_img_train.shape[-3:]) # decimal scaling

In [12]:
from tensorflow.keras.optimizers import Adam

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)

epoch=30

batch_size=64

lr=0.001

### Change here ###
costum_metric = False
# if you want ot use a tu use the costum loss function, type True
# else, False
### end

if costum_metric:
    metric, loss = cc.costum_mae, cc.costum_mse
else:
    metric, loss = 'mae', 'mse'


## MODELS

# Simple Neural Network
snn_model1.compile(optimizer = Adam(lr = lr), 
              loss = 'mse',
                   metrics = 'mae')

snn_model2.compile(optimizer = Adam(lr = lr), 
              loss = loss,
                   metrics = metric)

snn_model3.compile(optimizer = Adam(lr = lr), 
              loss = loss,
                   metrics = metric)

snn_model4.compile(optimizer = Adam(lr = lr), 
              loss = loss,
                   metrics = metric)

In [13]:
history_snn_model1, snn_model1= cc.fit_network(model=snn_model1, batchsize=batch_size, epoch=epoch, 
                                                           callback_properties=[True, False, 'no_norm.h5'],
                                                           x_img_train=x_img_train, x_ctg_train=x_ctg_train, y_train=y_train,
                                                           x_img_val=x_img_val, x_ctg_val=x_ctg_val, y_val=y_val)

Epoch 1/5
52/52 - 16s - loss: 282.7238 - mae: 11.9525 - val_loss: 286.3024 - val_mae: 12.0042

Epoch 00001: val_mae improved from inf to 12.00419, saving model to no_norm.h5
Epoch 2/5
52/52 - 6s - loss: 231.2777 - mae: 10.9066 - val_loss: 386.8003 - val_mae: 13.3552

Epoch 00002: val_mae did not improve from 12.00419
Epoch 3/5
52/52 - 6s - loss: 156.3790 - mae: 8.6180 - val_loss: 114.8075 - val_mae: 7.0457

Epoch 00003: val_mae improved from 12.00419 to 7.04574, saving model to no_norm.h5
Epoch 4/5
52/52 - 6s - loss: 75.7352 - mae: 5.6942 - val_loss: 43.4238 - val_mae: 4.3098

Epoch 00004: val_mae improved from 7.04574 to 4.30984, saving model to no_norm.h5
Epoch 5/5
52/52 - 6s - loss: 28.0844 - mae: 3.4296 - val_loss: 16.7473 - val_mae: 2.6341

Epoch 00005: val_mae improved from 4.30984 to 2.63411, saving model to no_norm.h5


In [14]:
history_snn_model2, snn_model2= cc.fit_network(model=snn_model2, batchsize=batch_size, epoch=epoch, 
                                                           callback_properties=[True, True, 'z_score.h5'],
                                                           x_img_train=x_img_train, x_ctg_train=x_ctg_train, y_train=y1_train,
                                                           x_img_val=x_img_val, x_ctg_val=x_ctg_val, y_val=y1_val)

Epoch 1/5
52/52 - 8s - loss: 15.5984 - costum_mae: 12.8368 - val_loss: 19.6249 - val_costum_mae: 12.4863

Epoch 00001: val_costum_mae improved from inf to 12.48634, saving model to z_score.h5
Epoch 2/5
52/52 - 6s - loss: 6.9210 - costum_mae: 8.4539 - val_loss: 8.5526 - val_costum_mae: 9.8400

Epoch 00002: val_costum_mae improved from 12.48634 to 9.84003, saving model to z_score.h5
Epoch 3/5
52/52 - 6s - loss: 5.6420 - costum_mae: 7.5742 - val_loss: 6.1330 - val_costum_mae: 8.3277

Epoch 00003: val_costum_mae improved from 9.84003 to 8.32772, saving model to z_score.h5
Epoch 4/5
52/52 - 6s - loss: 5.0202 - costum_mae: 7.0830 - val_loss: 4.8203 - val_costum_mae: 6.9648

Epoch 00004: val_costum_mae improved from 8.32772 to 6.96482, saving model to z_score.h5
Epoch 5/5
52/52 - 6s - loss: 4.5813 - costum_mae: 6.7021 - val_loss: 5.2009 - val_costum_mae: 7.2694

Epoch 00005: val_costum_mae did not improve from 6.96482


In [15]:
history_snn_model3, snn_model3= cc.fit_network(model=snn_model3, batchsize=batch_size, epoch=epoch, 
                                                           callback_properties=[True, True, 'min_max.h5'],
                                                           x_img_train=x_img_train, x_ctg_train=x_ctg_train, y_train=y2_train,
                                                           x_img_val=x_img_val, x_ctg_val=x_ctg_val, y_val=y2_val)

Epoch 1/5
52/52 - 8s - loss: 18.8514 - costum_mae: 14.0465 - val_loss: 178.5733 - val_costum_mae: 30.1102

Epoch 00001: val_costum_mae improved from inf to 30.11019, saving model to min_max.h5
Epoch 2/5
52/52 - 6s - loss: 5.8151 - costum_mae: 8.2089 - val_loss: 6.5236 - val_costum_mae: 7.8109

Epoch 00002: val_costum_mae improved from 30.11019 to 7.81094, saving model to min_max.h5
Epoch 3/5
52/52 - 6s - loss: 2.6815 - costum_mae: 5.5484 - val_loss: 2.4888 - val_costum_mae: 5.4815

Epoch 00003: val_costum_mae improved from 7.81094 to 5.48149, saving model to min_max.h5
Epoch 4/5
52/52 - 6s - loss: 1.5512 - costum_mae: 4.2645 - val_loss: 1.3551 - val_costum_mae: 4.1049

Epoch 00004: val_costum_mae improved from 5.48149 to 4.10486, saving model to min_max.h5
Epoch 5/5
52/52 - 6s - loss: 0.9236 - costum_mae: 3.2936 - val_loss: 1.0523 - val_costum_mae: 3.6519

Epoch 00005: val_costum_mae improved from 4.10486 to 3.65191, saving model to min_max.h5


In [16]:
history_snn_model4, snn_model4= cc.fit_network(model=snn_model4, batchsize=batch_size, epoch=epoch, 
                                                           callback_properties=[True, True, 'dec_scaling.h5'],
                                                           x_img_train=x_img_train, x_ctg_train=x_ctg_train, y_train=y3_train,
                                                           x_img_val=x_img_val, x_ctg_val=x_ctg_val, y_val=y3_val)


Epoch 1/5
52/52 - 8s - loss: 14.5560 - costum_mae: 12.1587 - val_loss: 33.1315 - val_costum_mae: 19.2657

Epoch 00001: val_costum_mae improved from inf to 19.26575, saving model to dec_scaling.h5
Epoch 2/5
52/52 - 6s - loss: 2.9938 - costum_mae: 5.6903 - val_loss: 5.3197 - val_costum_mae: 7.8453

Epoch 00002: val_costum_mae improved from 19.26575 to 7.84534, saving model to dec_scaling.h5
Epoch 3/5
52/52 - 6s - loss: 1.1756 - costum_mae: 3.6393 - val_loss: 2.4702 - val_costum_mae: 5.2641

Epoch 00003: val_costum_mae improved from 7.84534 to 5.26408, saving model to dec_scaling.h5
Epoch 4/5
52/52 - 6s - loss: 0.6482 - costum_mae: 2.7160 - val_loss: 1.5552 - val_costum_mae: 4.1477

Epoch 00004: val_costum_mae improved from 5.26408 to 4.14771, saving model to dec_scaling.h5
Epoch 5/5
52/52 - 6s - loss: 0.4330 - costum_mae: 2.2565 - val_loss: 0.8105 - val_costum_mae: 3.0254

Epoch 00005: val_costum_mae improved from 4.14771 to 3.02541, saving model to dec_scaling.h5


# Analysis

In [17]:
## Loading best weigths

snn_model1.load_weights('no_norm.h5')

snn_model2.load_weights('z_score.h5')

snn_model3.load_weights('min_max.h5')

snn_model4.load_weights('dec_scaling.h5')

# Training

In [18]:
df_skin, df_muscle, df_fat = cc.error_function(x_img_train, x_ctg_train, y_train, y_info,
                                               models=[snn_model1,snn_model2,snn_model3,snn_model4], categorical=True)

C:\Users\Acer\anaconda3\envs\tf2.4\lib\site-packages\tensorflow\python\keras\engine\functional.py:592: UserWarning: Input dict contained keys ['Extremity', 'Position'] which did not match any model input. They will be ignored by the model.
  warnings.warn(


finish no_normalization
finish z_score
finish min_max
finish decimal_scaling


In [19]:
columns=list(df_skin.columns)
mae_tissue={'skin': 0.36, 'muscle':0.78, 'fat':0.65}
std_tissue={'skin': 0.38, 'muscle':1.10, 'fat':1.11}
columns

['no_norm', 'z_score', 'min_max', 'decimal_scaling']

In [20]:
dfs={'skin':df_skin, 'fat':df_fat, 'muscle':df_muscle}
for name in dfs.keys():
    print (name)
    df=dfs[name]
    for i in columns:
        print('mae',i,': %.3f' %df[i].mean(),'std %.2f'%df[i].mad())
    print('mae IOE:',mae_tissue[name],'std',std_tissue[name])
    print('-'*45)

skin
mae no_norm : 0.350 std 0.20
mae z_score : 0.302 std 0.18
mae min_max : 0.519 std 0.32
mae decimal_scaling : 0.675 std 0.41
mae IOE: 0.36 std 0.38
---------------------------------------------
fat
mae no_norm : 2.725 std 1.92
mae z_score : 3.990 std 2.43
mae min_max : 8.321 std 4.78
mae decimal_scaling : 15.888 std 8.83
mae IOE: 0.65 std 1.11
---------------------------------------------
muscle
mae no_norm : 3.750 std 2.16
mae z_score : 3.800 std 2.45
mae min_max : 17.066 std 9.61
mae decimal_scaling : 15.592 std 9.90
mae IOE: 0.78 std 1.1
---------------------------------------------


# Validation

In [21]:
df_skin, df_muscle, df_fat = cc.error_function(x_img_val, x_ctg_val, y_val, y_info,
                                               models=[snn_model1,snn_model2,snn_model3,snn_model4], categorical=True)

finish no_normalization
finish z_score
finish min_max
finish decimal_scaling


In [22]:
dfs={'skin':df_skin, 'fat':df_fat, 'muscle':df_muscle}

for name in dfs.keys():
    print (name)
    df=dfs[name]
    for i in columns:
        print('mae',i,': %.3f' %df[i].mean(),'std %.2f'%df[i].mad())
    print('mae IOE:',mae_tissue[name],'std',std_tissue[name])
    print('-'*45)

skin
mae no_norm : 0.352 std 0.20
mae z_score : 0.297 std 0.18
mae min_max : 0.512 std 0.32
mae decimal_scaling : 0.720 std 0.42
mae IOE: 0.36 std 0.38
---------------------------------------------
fat
mae no_norm : 2.677 std 1.92
mae z_score : 4.141 std 2.47
mae min_max : 7.421 std 4.22
mae decimal_scaling : 16.949 std 9.17
mae IOE: 0.65 std 1.11
---------------------------------------------
muscle
mae no_norm : 4.873 std 2.68
mae z_score : 4.323 std 2.73
mae min_max : 17.557 std 9.45
mae decimal_scaling : 16.196 std 10.82
mae IOE: 0.78 std 1.1
---------------------------------------------


# Testing

In [23]:
df_skin, df_muscle, df_fat = cc.error_function(x_img_test, x_ctg_test, y_test,y_info,
                                               models=[snn_model1,snn_model2,snn_model3,snn_model4], categorical=True)

finish no_normalization
finish z_score
finish min_max
finish decimal_scaling


In [24]:
dfs={'skin':df_skin, 'fat':df_fat, 'muscle':df_muscle}
for name in dfs.keys():
    print (name)
    df=dfs[name]
    for i in columns:
        print('mae',i,': %.3f' %df[i].mean(),'std %.2f'%df[i].mad())
    print('mae IOE:',mae_tissue[name],'std',std_tissue[name])
    print('-'*45)

skin
mae no_norm : 0.410 std 0.21
mae z_score : 0.349 std 0.19
mae min_max : 0.559 std 0.33
mae decimal_scaling : 0.693 std 0.45
mae IOE: 0.36 std 0.38
---------------------------------------------
fat
mae no_norm : 2.767 std 1.98
mae z_score : 3.825 std 2.26
mae min_max : 8.249 std 4.82
mae decimal_scaling : 16.102 std 9.07
mae IOE: 0.65 std 1.11
---------------------------------------------
muscle
mae no_norm : 4.071 std 2.34
mae z_score : 3.989 std 2.38
mae min_max : 16.894 std 9.02
mae decimal_scaling : 15.838 std 10.44
mae IOE: 0.78 std 1.1
---------------------------------------------
